In [ ]:
#!pip install pdfminer3

In [9]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('Sentencias/sentencia_11966.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

print(text)
#text


MINISTERIO DE COMERCIO, INDUSTRIA Y TURISMO
SUPERINTENDENCIA DE INDUSTRIA Y COMERCIO
DELEGATURA PARA ASUNTOS JURISDICCIONALES

BOGOTÁ D.C., 9/11/2022

SENTENCIA NÚMERO 11966

Acción de Protección al Consumidor
Radicado No. 22- 17772
Demandante: MARIA AMPARO PARDO DE ORTIZ
Demandado: PEDRO JOSE ROA ORTEGA

Estando el expediente al Despacho a causa de encontrarse vencido el término de traslado de la
demanda, procede la Delegatura para Asuntos Jurisdiccionales a proferir sentencia escrita, teniendo
en cuenta que en el presente asunto se cumplen todos los presupuestos contenidos en el inciso 2º
del parágrafo 3º del artículo 390 del Código General del Proceso.

I.

ANTECEDENTES

1.

De la demanda

1.1.

Hechos

1.1.1. Adujo la parte demandante que adquirió del extremo demandado un juego de comedor de 6
puestos, como consta en la factura Nro. 0330 del 22082021, esto es, una mesa blanca con sillas
blancas tapizadas color azul, por la suma de UN MILLON TRESCIENTOS CINCUENTA MIL PESOS
M/CTE ($1